# Internet Access in New Orleans
This webpage was created by Jacob Masur and Daniel Cohen to investigate the distribution
of internet access in New Orleans across geographic regions. We would like to use geographic
data to understand how other factors such as race and income affect who has access to the
internet.


The first data set we will consider is sourced from NHGIS, it contains census tract data on the type of computers and the precense and types of internet subscriptions in households.  
Source:  
Steven Manson, Jonathan Schroeder, David Van Riper, Tracy Kugler, and Steven Ruggles.  
        IPUMS National Historical Geographic Information System: Version 16.0  
        [dataset]. Minneapolis, MN: IPUMS. 2021. 
        http://doi.org/10.18128/D050.V16.0

In [4]:
import pandas as pd
internet = pd.read_csv("../data/internet_data.csv")
internet.head()

,GISJOIN,YEAR,REGIONA,DIVISIONA,STATE,STATEA,COUNTY,COUNTYA,COUSUBA,PLACEA,...,AIQBM004,AIQBM005,AIQBM006,AIQBM007,AIQBM008,AIQBM009,AIQBM010,AIQBM011,AIQBM012,AIQBM013
0,G0100010020100,2013-2017,NaN,NaN,Alabama,1,Autauga County,1,NaN,NaN,...,71,69,36,75,61,30,8,11,8,50
1,G0100010020200,2013-2017,NaN,NaN,Alabama,1,Autauga County,1,NaN,NaN,...,85,80,52,85,51,20,10,11,24,82
2,G0100010020300,2013-2017,NaN,NaN,Alabama,1,Autauga County,1,NaN,NaN,...,129,107,81,129,86,57,17,11,14,108
3,G0100010020400,2013-2017,NaN,NaN,Alabama,1,Autauga County,1,NaN,NaN,...,160,142,78,158,107,61,11,11,25,126
4,G0100010020500,2013-2017,NaN,NaN,Alabama,1,Autauga County,1,NaN,NaN,...,372,362,174,434,200,92,45,16,148,183


This dataset contains data from accross the country, but we are only interested in Orleans Parish data.

In [9]:
internet = internet[(internet["STATE"] == "Louisiana") & (internet["COUNTY"] == "Orleans Parish")]
internet.head()

,GISJOIN,YEAR,REGIONA,DIVISIONA,STATE,STATEA,COUNTY,COUNTYA,COUSUBA,PLACEA,...,AIQBM004,AIQBM005,AIQBM006,AIQBM007,AIQBM008,AIQBM009,AIQBM010,AIQBM011,AIQBM012,AIQBM013
28917,G2200710000100,2013-2017,NaN,NaN,Louisiana,22,Orleans Parish,71,NaN,NaN,...,155,144,75,145,129,79,12,12,34,87
28918,G2200710000200,2013-2017,NaN,NaN,Louisiana,22,Orleans Parish,71,NaN,NaN,...,79,42,11,79,72,6,12,12,45,47
28919,G2200710000300,2013-2017,NaN,NaN,Louisiana,22,Orleans Parish,71,NaN,NaN,...,56,51,41,46,33,15,9,12,10,68
28920,G2200710000400,2013-2017,NaN,NaN,Louisiana,22,Orleans Parish,71,NaN,NaN,...,80,61,43,62,61,34,29,12,38,84
28921,G2200710000601,2013-2017,NaN,NaN,Louisiana,22,Orleans Parish,71,NaN,NaN,...,43,34,9,43,31,7,12,12,14,71


Most of the columns in the data set are irrelevant to our investigation, so we will drop many of them.

In [12]:
internet

,GISJOIN,YEAR,REGIONA,DIVISIONA,STATE,STATEA,COUNTY,COUNTYA,COUSUBA,PLACEA,...,AIQBM004,AIQBM005,AIQBM006,AIQBM007,AIQBM008,AIQBM009,AIQBM010,AIQBM011,AIQBM012,AIQBM013
28917,G2200710000100,2013-2017,NaN,NaN,Louisiana,22,Orleans Parish,71,NaN,NaN,...,155,144,75,145,129,79,12,12,34,87
28918,G2200710000200,2013-2017,NaN,NaN,Louisiana,22,Orleans Parish,71,NaN,NaN,...,79,42,11,79,72,6,12,12,45,47
28919,G2200710000300,2013-2017,NaN,NaN,Louisiana,22,Orleans Parish,71,NaN,NaN,...,56,51,41,46,33,15,9,12,10,68
28920,G2200710000400,2013-2017,NaN,NaN,Louisiana,22,Orleans Parish,71,NaN,NaN,...,80,61,43,62,61,34,29,12,38,84
28921,G2200710000601,2013-2017,NaN,NaN,Louisiana,22,Orleans Parish,71,NaN,NaN,...,43,34,9,43,31,7,12,12,14,71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29089,G2200710014400,2013-2017,NaN,NaN,Louisiana,22,Orleans Parish,71,NaN,NaN,...,74,61,25,74,59,12,12,12,30,68
29090,G2200710014500,2013-2017,NaN,NaN,Louisiana,22,Orleans Parish,71,NaN,NaN,...,12,12,12,12,12,12,12,12,12,12
29091,G2200710980000,2013-2017,NaN,NaN,Louisiana,22,Orleans Parish,71,NaN,NaN,...,12,12,12,12,12,12,12,12,12,12
29092,G2200710980100,2013-2017,NaN,NaN,Louisiana,22,Orleans Parish,71,NaN,NaN,...,12,12,12,12,12,12,12,12,12,12
